In [ ]:
import numpy as np
import matplotlib.pyplot as plt

We can add an additional process to compute the geometry of the water table assuming that it obeys the Dupuit-Forchheimer assumptions that flow is dominantly lateral and that discharge is proportional to the saturated aquifer thickness:
$$\frac{\partial}{\partial x}K(H(x)-z(x)+B(x))\frac{\partial H}{\partial x}+P(x)=0 $$
Numerically, as shown in Braun et al (2016), this can be solved in 1D using an $O(n)$, second-order accurate algorithm:
$$\frac{K}{2}(H_i-z_i+B_i+H_{i-1}-z_{i-1}+B_{i-1})\frac{H_i-H_{i-1}}{\Delta x }=F_i $$
where:
$$F_{i-1}=F_i+P_{i-1}\Delta x$$

In [ ]:
import xsimlab as xs
import xarray as xr
%load_ext xsimlab.ipython

In [ ]:
@xs.process
class Topography:
    elevation = xs.variable(dims='x', intent='inout',
                      description='topography elevation',
                      attrs={'units': 'm'})
    dtopo = xs.group('dtopo')
    
    def run_step(self):
        self.elevation = self.elevation + sum(self.dtopo)
        self.elevation[-1] = self.elevation[-2]

In [ ]:
@xs.process
class Erosion:
    
    kd = xs.variable(dims=[(),'x'],
                     description='transport coefficient',
                    attrs={'units': 'm^2/yr'})
    surface = xs.foreign(Topography, 'elevation', intent='in')
    L = xs.variable(description='length',
                   attrs={'units': 'm'})
    dx = xs.variable(intent='out',
                    description='spatial step',
                    attrs={'units': 'm'})
    dtopo = xs.variable(dims='x', intent='out',
                        description='erosional increment in topography',
                        attrs={'untis': 'm'},
                        groups='dtopo')
    
    def initialize(self):
        self.dx = self.L/(len(self.surface) - 1)
        self.dtopo = np.zeros_like(self.surface)
        
    @xs.runtime(args=("step_delta"))
    def run_step(self, dt):
        kd_var = np.broadcast_to(self.kd, self.surface.shape).flatten()
        self.dtopo[1:-1] = (dt/self.dx**2/2
                           *((self.surface[:-2] - self.surface[1:-1])*(kd_var[:-2]+kd_var[1:-1])
                            -(self.surface[1:-1] - self.surface[2:])*(kd_var[1:-1]+kd_var[2:])))
        self.dtopo[0] = 0
        self.dtopo[-1] = self.dtopo[-2]


In [ ]:
@xs.process
class Geom:
    length = xs.variable(description='length',
                        attrs={'units': 'm'})
    slope = xs.variable(description='initial slope')
    nx = xs.variable(description='spatial resolution')
    L = xs.foreign(Erosion, 'L', intent='out')
    z = xs.foreign(Topography, 'elevation', intent='out')
    
    def initialize(self):
        self.L = self.length
        self.z = np.linspace(0, self.length, self.nx)


In [ ]:
@xs.process
class Uplift:
    rate = xs.variable(dims=[(),'x'],
                       description='uplift rate',
                      attrs={'units': 'm/yr'})
    dtopo = xs.variable(dims='x', intent='out',
                        description='uplift increment in topography',
                        attrs={'untis': 'm'},
                       groups='dtopo')
    nx = xs.foreign(Geom, 'nx')
    
    @xs.runtime(args=("step_delta"))
    def run_step(self, dt):
        u_var = np.broadcast_to(self.rate, self.nx).flatten()
        self.dtopo = dt*u_var
        self.dtopo[0] = 0


For this we add a process to integrate precipitation

In [ ]:
def integrate(x):
    """
    integrate:
    ---------

    Function to integrate variable x by simple summation:
    int_i = sum_(j = 1 to i) of x_j

    in input:
    x: array to integrate

    in output:
    integ: result of the integration

    """

    integ = x.copy()
    for i in range(len(integ) - 1, 0, -1):
        integ[i - 1] = integ[i - 1] + integ[i]

    return integ

As well as another one to solve the hydraulic equation

In [ ]:
def computeHydro(B, z, accum, dx, K):
    """
    table:
    -----

    Function to compute the geometry of the water table
    using the second-order accurate finite difference scheme
    described in Braun et al (2015)

    in input:
    B: the thickness of the regolith
    z: the surface topography
    accum: cumulative infiltration rate
    dx: distance between two nodes
    K: hydraulic conductivity


    in output:
    H: water table height

    """

    H = z.copy()
    for i in range(1, len(H)):
        b = -z[i] - z[i - 1] + B[i] + B[i - 1]
        c = (z[i] + z[i - 1] - B[i] - B[i - 1] - H[i - 1]) * H[i - 1] - 2 * dx * accum[
            i
        ] / K
        H[i] = (-b + np.sqrt(b ** 2 - 4 * c)) / 2
        if H[i] > z[i]:
            H[i] = z[i]
        if H[i] < z[i] - B[i]:
            H[i] = z[i] - B[i]
    return H

We now build a more complex model that has up to six processes

And we can predict the geometry of the water table under an actively uplifting and eroding hill